In [54]:
import conllu
import pandas as pd
from tqdm import tqdm
import numpy as np

http://www.nilc.icmc.usp.br/semanticnlp/index.php?id=index&id_sub=principal&dir_sub=includes/projects/propbankbr&dir=includes/projects/propbankbr&lang=pt-br#palmeretal2005

ID: Identificador do token na sentença. (Ex.: 1, 2, 3, ...)

FORM: Forma do token no texto. (Ex.: Brasília, Pesquisa_Datafolha)

LEMMA: Lema ou forma canônica do token. (Ex.: Brasília, Pesquisa_Datafolha)

CPOSTAG: Tag de parte do discurso grosseira (coarse-grained part-of-speech tag). (Ex.: PROP, N, V-PCP)

POSTAG: Tag de parte do discurso detalhada (fine-grained part-of-speech tag). (Ex.: F|S, PR|3S|IND)

FEATS: Características morfológicas do token. (Ex.: (S*, (FCL*)

HEAD: Cabeça do token (geralmente um identificador de outro token ao qual 
este está ligado sintaticamente). (Ex.: (FCL*)

DEPREL: Relação de dependência entre o token e sua cabeça. (Ex.: (FCL(NP*)

PHEAD: Cabeça de projeto (para anotações projetadas ou sugeridas). (Ex.: -)

PDEPREL: Relação de dependência de projeto. (Ex.: -)

SRL: Anotação de papel semântico, que identifica o papel do token em relação ao verbo principal (Ex.: (A0*)

Coref: Informação de co-referência, que conecta entidades referidas de maneiras diferentes ao longo do texto. (Ex.: (AM-ADV*)

In [55]:
path = '../data/raw/propbankbr_v1.1_conll/PropBankBr_v1.1_Const.conll.txt'

# Open the plain text file for reading; assign under 'data'
with open(path, mode="r", encoding="utf-8") as data:
    
    # Read the file contents and assign under 'annotations'
    annotations = data.read()

In [56]:
sents = annotations.split("\n\n")
sents = [sent for sent in sents if sent != '']

In [57]:
list_dict_an = []
propbank_id = 0
for sent in tqdm(sents):
    
    propbank_id += 1
    
    dict_an = {}

    tks = sent.split("\n")
    
    list_id = []
    list_form = []
    list_lemma = []
    list_cpostag = []
    list_postag = []
    list_feats = []
    list_head = []
    list_deprel = []
    list_phead = []
    list_pdeprel = []

    args_list = []
    
    
    dict_an_snt = {}
    for id_tk, tk in enumerate(tks):

        tk_an = tk.split("\t")

        # pega as 10 caracteristicas (colunas) fixas (as restantes sao args)
        id, form, lemma, cpostag, postag, feats, head, deprel, phead, pdeprel = tk_an[:10]
        
        id = int(id)
        form = form.replace(" ", "")
        lemma = lemma.replace(" ", "")
        cpostag = cpostag.replace(" ", "") 
        postag = postag.replace(" ", "")
        feats = feats.replace(" ", "")
        head = head.replace(" ", "") 
        deprel = deprel.replace(" ", "") 
        phead = phead.replace(" ", "")
        pdeprel = pdeprel.replace(" ", "")
        
        
        list_id.append(id)
        list_form.append(form)
        list_lemma.append(lemma)
        list_cpostag.append(cpostag)
        list_postag.append(postag) 
        list_feats.append(feats) 
        list_head.append(head) 
        list_deprel.append(deprel) 
        list_phead.append(phead) 
        list_pdeprel.append(pdeprel) 

        # get args 
        args = tk_an[10:len(tk_an)]
        # format  
        args = [arg.replace(" ", "")  for arg in args]
        
        
        
        args_list.append(np.array(args))
        
        dict_an_snt.update({
            id_tk:{
                "id": id,
                "form": form,
                "lemma": lemma,
                "cpostag": cpostag, 
                "postag": postag, 
                "feats": feats, 
                "head": head, 
                "deprel": deprel, 
                "phead": phead, 
                "pdeprel": pdeprel
            }
        })
    
    sentence = " ".join(list_form)
    
    
    dict_an.update({"id": "propbank" + "_" + str(propbank_id)})
    dict_an.update({"snt":sentence})
    dict_an.update({"annotated_sentence_gold":dict_an_snt})
    list_dict_an.append(dict_an)
    
    
        
        
    args_mtx = np.array(args_list)
    args_df = pd.DataFrame(args_mtx, columns = [f'level{i}' for i in range(args_mtx.shape[1])], index=list_id)
    # replace * with nan (all the values are ARGS or V now)
    args_df.replace('*', np.nan, inplace=True)
    
    

100%|██████████| 3348/3348 [00:01<00:00, 2400.91it/s]


In [58]:
for col in args_df.columns:
    
    # drop tokens that do not have args
    args_series = args_df[col].dropna()
    
    print()

In [59]:
args_series

9         (V*)
10        (A1*
14          *)
15    (AM-TMP*
20          *)
Name: level1, dtype: object

In [60]:
dict_an

{'id': 'propbank_3348',
 'snt': 'Além_de Mauro_Salles que surpreendeu a galera a o revelar seu côté de fotógrafo profissional em o ínico de a carreira , o foco principal de o encontro foi a mulher de J.R.Duran , Alexandra_Brochen .',
 'annotated_sentence_gold': {0: {'id': 1,
   'form': 'Além_de',
   'lemma': 'além_de',
   'cpostag': 'PRP',
   'postag': '-',
   'feats': '(S*',
   'head': '(FCL*',
   'deprel': '(FCL(PP*',
   'phead': '-',
   'pdeprel': '-'},
  1: {'id': 2,
   'form': 'Mauro_Salles',
   'lemma': 'Mauro_Salles',
   'cpostag': 'PROP',
   'postag': 'M|S',
   'feats': '*',
   'head': '*',
   'deprel': '(NP*',
   'phead': '-',
   'pdeprel': '-'},
  2: {'id': 3,
   'form': 'que',
   'lemma': 'que',
   'cpostag': 'PRON-INDP',
   'postag': 'M|S',
   'feats': '(S*',
   'head': '(FCL*',
   'deprel': '(FCL(NP*)',
   'phead': '-',
   'pdeprel': '-'},
  3: {'id': 4,
   'form': 'surpreendeu',
   'lemma': 'surpreender',
   'cpostag': 'V-FIN',
   'postag': 'PS|3S|IND',
   'feats': '*',
 